In [76]:
#Needed libraries installation (run it once)
#%pip install -r requirements.txt

In [77]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.1", request_timeout=4000.0)

In [79]:
from llama_index.core import SimpleDirectoryReader


documents = SimpleDirectoryReader("data").load_data()



In [80]:
from transformers import AutoModel, AutoTokenizer

save_directory = "./local_embedding_model"

In [ ]:
# # Download local embedding model, should be ran only once

# model_name = "BAAI/bge-small-en-v1.5"


# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Save to local directory

# tokenizer.save_pretrained(save_directory)
# model.save_pretrained(save_directory)

In [82]:
# Load local model 
# Load from local directory
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModel.from_pretrained(save_directory)

In [83]:
#create embedding function
from llama_index.core.embeddings import BaseEmbedding
import torch

class LocalEmbedding(BaseEmbedding):
    def __init__(self, model, tokenizer):
        super().__init__()
        self._model = model  # Explicitly define with a private attribute
        self._tokenizer = tokenizer

    def get_text_embedding(self, text):
        # Tokenize with truncation to the model's maximum length
        inputs = self._tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        with torch.no_grad():
            outputs = self._model(**inputs)
        # Compute mean of the last hidden state as embedding
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        return embedding

    # Implement required methods
    def _get_query_embedding(self, query):
        return self.get_text_embedding(query)

    def _get_text_embedding(self, text):
        return self.get_text_embedding(text)

    async def _aget_query_embedding(self, query):
        return self.get_text_embedding(query)

# Initialize custom embedding with your model and tokenizer
embedding_model = LocalEmbedding(model=model, tokenizer=tokenizer)


In [84]:
from llama_index.core import VectorStoreIndex, Settings


# Configure global settings
Settings.llm = llm
Settings.embed_model = embedding_model
# Set the chunk size (number of tokens per chunk)
Settings.chunk_size = 256
# Set the chunk overlap (number of tokens overlapping between chunks)
Settings.chunk_overlap = 64

index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 50/50 [00:07<00:00,  7.06it/s]


In [85]:
# Set up the retriever from your index
top_k = 5
retriever = index.as_retriever(similarity_top_k=top_k)


In [86]:
def get_augmented_prompt(query):
    retrieved_documents = retriever.retrieve(query)
    # Augment the prompt by including the retrieved documents
    # Constructing an augmented prompt with context from retrieved documents
    augmented_prompt = f"Context:\n"
    for index, doc in enumerate(retrieved_documents):
        text = doc.text
        print(f"Retrived {index+1}: {text} \n")
        augmented_prompt += f"{text}\n"
    augmented_prompt += f"\nQuestion: {query}\nPlease provide a concise and accurate answer based on the context."
    return augmented_prompt


In [87]:
from IPython.display import Markdown, display

# Send the augmented prompt to a language model for a synthesized response
def get_RAG_response(query):    
    response = llm.complete(get_augmented_prompt(query))
    return response


In [88]:
query = "What is the Bayraktar?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))


Retrived 1: Lowther & SiddikiAIR & SPACE OPERATIONS REVIEW   5defense technologies and began to modernize the military.10 These efforts included coop -
erating with many Western nations to acquire state- of- the- art military hardware and 
software. In this regard, Ukraine’s homegrown military drone technology was relatively 
young when Russia invaded in February 2022.11 Many drones Ukraine has deployed 
against Russia were purchased from foreign nations, with Turkey leading the list of sup -
pliers before the start of the war.
Baykar Bayraktar TB2
The Baykar Bayraktar TB2 is a medium- altitude, long- endurance unmanned combat 
aerial vehicle that, according to the manufacturer, is capable of remotely controlled or 
autonomous flight operations. This immensely popular drone is manufactured by the 
Turkish defense company Baykar Defense.12 The manufacturer’s primary objective is to 
build a less expensive alternative to Western drones, primarily for the Turkish Armed 
Forces. 

Retrived

## <b>Response:</b> 
 #### The Bayraktar TB2 is a medium-altitude, long-endurance unmanned combat aerial vehicle (drone) manufactured by the Turkish defense company Baykar Defense. It is a less expensive alternative to Western drones, primarily designed for the Turkish Armed Forces, but has proven effective in various conflicts, including Ukraine's war against Russia.

In [89]:
query = "Where was used Bayraktar?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

Retrived 1: Lowther & SiddikiAIR & SPACE OPERATIONS REVIEW   5defense technologies and began to modernize the military.10 These efforts included coop -
erating with many Western nations to acquire state- of- the- art military hardware and 
software. In this regard, Ukraine’s homegrown military drone technology was relatively 
young when Russia invaded in February 2022.11 Many drones Ukraine has deployed 
against Russia were purchased from foreign nations, with Turkey leading the list of sup -
pliers before the start of the war.
Baykar Bayraktar TB2
The Baykar Bayraktar TB2 is a medium- altitude, long- endurance unmanned combat 
aerial vehicle that, according to the manufacturer, is capable of remotely controlled or 
autonomous flight operations. This immensely popular drone is manufactured by the 
Turkish defense company Baykar Defense.12 The manufacturer’s primary objective is to 
build a less expensive alternative to Western drones, primarily for the Turkish Armed 
Forces. 

Retrived

## <b>Response:</b> 
 #### According to the text, the Bayraktar TB2 drone was used in:

1. Syria
2. Libya
3. Azerbaijan
4. Ukraine (specifically against Russian tanks and armored vehicles)

Additionally, it is mentioned that the Bayraktar TB2S was upgraded to the standard TB2 as part of Ukraine's military modernization program in 2019.

In [90]:

query = "Where was used Bayraktar?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

Retrived 1: Lowther & SiddikiAIR & SPACE OPERATIONS REVIEW   5defense technologies and began to modernize the military.10 These efforts included coop -
erating with many Western nations to acquire state- of- the- art military hardware and 
software. In this regard, Ukraine’s homegrown military drone technology was relatively 
young when Russia invaded in February 2022.11 Many drones Ukraine has deployed 
against Russia were purchased from foreign nations, with Turkey leading the list of sup -
pliers before the start of the war.
Baykar Bayraktar TB2
The Baykar Bayraktar TB2 is a medium- altitude, long- endurance unmanned combat 
aerial vehicle that, according to the manufacturer, is capable of remotely controlled or 
autonomous flight operations. This immensely popular drone is manufactured by the 
Turkish defense company Baykar Defense.12 The manufacturer’s primary objective is to 
build a less expensive alternative to Western drones, primarily for the Turkish Armed 
Forces. 

Retrived

## <b>Response:</b> 
 #### The Bayraktar TB2 drone was used in:

* Syria
* Libya
* Azerbaijan
* Ukraine (particularly during the Russian invasion in February 2022)

In [93]:
query = "Ukraine’s Drones. Give me the list of the drone's names. Exclude russian's drones"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

Retrived 1: Russia has not disclosed how many Orion drones it operates in Ukraine, but it has 
claimed the drone has been used to attack Ukrainian positions successfully. Simultane -
ously, online photographic evidence shows the Ukrainians shot down at least one Orion- E.48
As the previous discussion of Russian drones in Ukraine suggests, Russia’s claims that 
it fields state- of- the- art drones are certainly questionable.49 To suggest Russia lacks 
dominance may be a generous description of the state of Russian capabilities. Simply 
stated, Russian drones are rarely in evidence in Ukraine. This failure is leading to specula -
tion among analysts that follow the war in Ukraine that Russian drones are being held in 
reserve for a later escalation in the conflict.50
Others suggest logistical issues constrain drone use—as evidenced by the widespread 
reports of abandoned and broken Russian military vehicles.51 This is prompting some to 
conclude Russia may not be able to support drone op

## <b>Response:</b> 
 #### Based on the provided text, here is the list of Ukrainian drones mentioned:

1. Bayraktar TB2 (imported)
2. Warmate (domestically produced)
3. Punisher (mid-size drone, domestically produced)

Note that there are also mentions of other drones being used by Ukraine, such as those developed by UA Dynamics and potentially others from Chinese manufacturer DJI, but the above list only includes the specific drones mentioned in the text.